## Import and Functions


In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error
from math import sqrt
import os
from numpy import dot
from numpy import sum, tile, linalg, log, pi, exp
from glob import glob
from numpy.linalg import inv, det
import math as m

In [19]:
def read_file(filename):
    return pd.read_csv(filename)

def get_data(timeframe):
    return read_file(r"C:\Users\ayush\Desktop\IITB\ZeltaLabPS\BackTesting\data\data\btcusdt_" + timeframe + "_total.csv")

time_frame = "1h"
# Load data
df = get_data(time_frame)
df.head()

,datetime,open,high,low,close,volume
0,2018-01-01 05:30:00,13715.65,13715.65,13400.01,13529.01,443.356199
1,2018-01-01 06:30:00,13528.99,13595.89,13155.38,13203.06,383.697006
2,2018-01-01 07:30:00,13203.00,13418.43,13200.00,13330.18,429.064572
3,2018-01-01 08:30:00,13330.26,13611.27,13290.00,13410.03,420.087030
4,2018-01-01 09:30:00,13434.98,13623.29,13322.15,13601.01,340.807329


In [20]:
def calculate_momentum(df, n=10):
    df['Momentum'] = df['close'] - df['close'].shift(n)
    df['Momentum'] = df['Momentum'] / df['close'].shift(n) * 100
    return df

def calculate_roc(df, n=10):
    df['ROC'] = (df['close'] - df['close'].shift(n)) / df['close'].shift(n)
    return df

def calculate_cci(df, n=20, factor=0.0015):
    typical_price = (df['high'] + df['low'] + df['close']) / 3
    sma = typical_price.rolling(window=n).mean()
    mean_deviation = (typical_price - sma).abs().rolling(window=n).mean()
    
    df['CCI'] = (typical_price - sma) / (factor * mean_deviation)
    
    return df

In [21]:
df['change_5'] = df['close'].rolling(window=5).mean().pct_change() * 1e4

df['ret_5'] = df['close'].rolling(window=5).mean().pct_change() * 1e4
df['ret_1'] = df['close'].rolling(window=1).mean().pct_change() * 1e4
df['ret_10'] = df['close'].rolling(window=10).mean().pct_change() * 1e4
df['ret_30'] = df['close'].rolling(window=30).mean().pct_change() * 1e4

df['ret_5'] = df['ret_5'].shift(-5)
df['ret_1'] = df['ret_1'].shift(-1)
df['ret_10'] = df['ret_10'].shift(-10)
df['ret_30'] = df['ret_30'].shift(-30)

df = df.dropna()
df = df.reset_index(drop=True)

In [22]:
def print_buckets(df, alphas, rets = ['ret_10', 'ret_30', 'ret_60', 'ret_300', 'tick_10'], aggfunc = ['mean', 'median', 'count'], buckets = 10):
# def print_buckets(df, alphas, rets = ['ret_10', 'ret_30', 'iv_10'], aggfunc = ['mean', 'median', 'count']):
# def print_buckets(df, alphas, rets = ['tick_10', 'tick_30', 'tick_60', 'tick_300'], aggfunc = ['mean', 'median', 'count']):
# def print_buckets(df, alphas, rets = ['ret_60', 'ret_300', 'iv_60'], aggfunc = ['mean', 'median', 'count']):
    for alpha in alphas:
        msg = df.pivot_table(index = pd.qcut(df[alpha],buckets,duplicates='drop'), values=rets, aggfunc=aggfunc)
        print(msg)
        
def print_corrs(df, alphas, rets = ['ret_10', 'ret_30', 'ret_60', 'ret_300']):
# def print_corrs(df, alphas, rets = ['ret_10', 'ret_30', 'iv_10']):
# def print_corrs(df, alphas, rets = ['tick_10', 'tick_30', 'tick_60', 'tick_300']):
# def print_corrs(df, alphas, rets = ['ret_60', 'ret_300', 'iv_60']):
    msg = "                                      "
    for ret in rets:
        msg += f"{ret:>8s}"
    print(msg)
    for alpha in alphas:
        msg = f"{alpha:30s} corr -> "
        for ret in rets:
            msg += f"{df[alpha].corr(df[ret])*100:7.2f} "
        print(msg)
    print()

In [23]:
def calculate_volatility(df, window=20):
    """
    Calculate historical volatility using daily returns.
    
    Parameters:
    - df: DataFrame with 'Close' prices
    - window: Rolling window size for volatility calculation
    
    Returns:
    - DataFrame with 'Volatility' column
    """
    # Calculate daily returns
    df['Daily_Return'] = df['close'].pct_change()

    # Calculate historical volatility using the rolling standard deviation
    df['Volatility'] = df['Daily_Return'].rolling(window=window).std() * (252**0.5)  # 252 trading days in a year

    return df.dropna()

In [24]:
df = calculate_volatility(df)
df = calculate_momentum(df)
df = calculate_roc(df)
df = calculate_cci(df)


In [25]:
def calculate_alpha(btc_data):
    # Ensure that the data is sorted in ascending order by date
    # btc_data = btc_data.sort_index()

    # Calculate the 5-day delta of close prices
    btc_data['Close_5D_Delta'] = btc_data['close'].diff(5)

    # Calculate the 20-day standard deviation of close prices
    btc_data['Close_20D_STD'] = btc_data['close'].rolling(window=20).std()

    # Calculate the 10-day correlation between close prices and volume
    btc_data['Close_Volume_10D_Corr'] = btc_data['close'].rolling(window=10).corr(btc_data['volume'])

    # Combine the calculated values into the alpha expression
    btc_data['Alpha'] = btc_data['Close_5D_Delta'] / btc_data['Close_20D_STD'] + btc_data['Close_Volume_10D_Corr']

    return btc_data

In [26]:
df = calculate_alpha(df)

## Checking direction with pivot

In [27]:
df['oc'] = df['close'] - df['open']


In [28]:
rets = ['ret_1', 'ret_5','ret_10']
aggfunc = ['mean', 'median', 'count']
# aggfunc = ['mean']

# alphas = ['k_alpha1', 'k_alpha2', 'k_alpha3', 'k_alpha4', 'k_alpha5']

In [29]:
msg = df.pivot_table(index = pd.qcut(df['oc'],10,duplicates='drop'), values=rets, aggfunc=aggfunc)


In [32]:
print_corrs(df, ['oc'], rets)

                                         ret_1   ret_5  ret_10
oc                             corr ->   -2.14   -3.06   -1.63 



In [30]:
print(msg)

                          mean                         median            \
                         ret_1    ret_10     ret_5      ret_1    ret_10   
oc                                                                        
(-3789.751, -88.742]  8.489002  1.555019  3.236612  11.627728  2.912970   
(-88.742, -34.72]     3.314755  0.373831  0.610431  10.538232  2.087365   
(-34.72, -16.97]     -1.577730  0.052066  0.340732   5.352195  0.699244   
(-16.97, -6.7]        1.216617 -0.275579 -0.152445   4.337784  0.647217   
(-6.7, 0.87]          0.966487  0.718529  1.127802   2.065284  0.752898   
(0.87, 8.02]          1.167018  0.056051 -0.047834  -0.166155  0.246622   
(8.02, 18.404]       -1.015978  0.082236  0.022152  -1.601571  0.133296   
(18.404, 37.18]      -1.629010  0.887848  0.851168  -4.756511  0.338883   
(37.18, 94.434]      -3.779703  0.247244 -0.901040  -9.243934 -0.187897   
(94.434, 3973.88]     0.092878 -0.047475 -1.084638  -2.410298  0.534749   

                        

In [31]:
print_buckets(df, ['Alpha'], rets, aggfunc, 10)

                      mean                        median                      \
                     ret_1    ret_10     ret_5     ret_1    ret_10     ret_5   
Alpha                                                                          
(-5.987, -2.24]   1.273004 -1.133195 -0.469645  7.066425  1.428727  2.771861   
(-2.24, -1.298]   4.235403  0.989589  3.095704  5.982316  2.265757  2.766610   
(-1.298, -0.72]   1.050310  0.259974  0.684186  4.912415  1.783830  2.770392   
(-0.72, -0.314]   2.041433 -0.564930 -0.280760  3.414342  0.597823  1.499200   
(-0.314, 0.0183]  2.192906  0.532381  0.203176  2.729709  0.486858  1.078914   
(0.0183, 0.345]  -0.984555 -0.069876 -0.296261 -0.069190  0.119381  0.341567   
(0.345, 0.743]   -4.201766  0.051577 -0.807117 -2.303657  0.145002  0.033021   
(0.743, 1.305]   -3.621245 -0.239523 -1.922460 -3.847672 -0.273185 -1.866025   
(1.305, 2.166]    1.176796  0.275310 -0.141966 -1.355300 -0.194873 -1.186120   
(2.166, 6.209]    3.808318  3.200663  3.